In [17]:
# call data.fif
import mne
import numpy as np

raw = mne.io.read_raw_fif('data.fif', preload=True)

new_annot = mne.Annotations(onset=[10, 20, 30], duration=[1, 2, 3], description=['foo', 'bar', 'baz']) # create new annotations

raw.set_annotations(new_annot) # set new annotations

Opening raw data file data.fif...
    Range : 0 ... 36909 =      0.000 ...    73.818 secs
Ready.
Reading 0 ... 36909  =      0.000 ...    73.818 secs...


C:\Users\FARHAN FRANAKA\AppData\Local\Temp\ipykernel_21212\1581367519.py:5: RuntimeWarning: This filename (data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif('data.fif', preload=True)


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,7 points
Good channels,"4 EEG, 1 Stimulus"
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,500.00 Hz
Highpass,0.00 Hz
Lowpass,250.00 Hz


In [ ]:
events, _ = mne.events_from_annotations(raw)
print(events)